In [2]:
import trimesh
import plotly.graph_objects as go
import numpy as np


def visualize_meshes_plotly(
    mesh_list,
    color_list=None,
    vertices_list = None,
    vertices_color_list = None,
    vertex_marker_size = 2,
    show_edges = True,
    edge_width = 2,
    show = True,
):
    # Pre-convert to list and load meshes once
    mesh_list = [mesh_list] if not isinstance(mesh_list, list) else mesh_list
    final_mesh_list = [trimesh.load(m) if isinstance(m, str) else m for m in mesh_list]
    
    color_list = color_list or ['lightgray'] * len(final_mesh_list)
    
    # Create all mesh traces at once
    mesh_traces = []
    edge_traces = []
    for mesh, color in zip(final_mesh_list, color_list):
        face_colors = mesh.visual.face_colors[:, :3] if hasattr(mesh.visual, 'face_colors') else None
        mesh_traces.append(go.Mesh3d(
            x=mesh.vertices[:, 0],
            y=mesh.vertices[:, 1],
            z=mesh.vertices[:, 2],
            i=mesh.faces[:, 0],
            j=mesh.faces[:, 1],
            k=mesh.faces[:, 2],
            opacity=0.5,
            facecolor=face_colors,
            color=None if face_colors is not None else color
        ))
        
        if show_edges:
            edge_x = []
            edge_y = []
            edge_z = []
            vertices = mesh.vertices
            for edge in mesh.edges:
                edge_x.extend([vertices[edge[0], 0], vertices[edge[1], 0], None])
                edge_y.extend([vertices[edge[0], 1], vertices[edge[1], 1], None])
                edge_z.extend([vertices[edge[0], 2], vertices[edge[1], 2], None])
        
            edge_traces.append(go.Scatter3d(
                x=edge_x, y=edge_y, z=edge_z,
                mode='lines',
                line=dict(
                    color = color if color is not None else 'red',
                    width=edge_width
                ),
                name='Edges'
            ))
    
    fig = go.Figure(data = mesh_traces + edge_traces)
    
    if vertices_list is not None and vertices_color_list is not None:
        for vertex, color in zip(vertices_list, vertices_color_list):
            fig.add_trace(go.Scatter3d(
                x=vertex[:, 0],
                y=vertex[:, 1],
                z=vertex[:, 2],
                mode='markers',
                marker=dict(size=vertex_marker_size, color=color, opacity=1),
                name='Vertices'
            ))
    fig.update_layout(
        scene=dict(aspectmode='data'),
        width=800,
        height=800,
        showlegend=False
    )
    
    if show:
        fig.show()
    return fig


In [10]:
mesh = trimesh.load(
    "smplh_pose_0100.obj"
)
print(mesh.vertices.shape, mesh.faces.shape)

fig = visualize_meshes_plotly(
    [mesh],
    show_edges=False,
)

(25130, 3) (13776, 3)


In [ ]:
FBX_PATH = "C:\Users\dytpq0916\VTO2025\REFERENCES\SewFormerAnalysis\SewFactory\examples\results\Subject_23_F_18_poses_40_SMPLH_female_010_207_animated.fbx"

import bpy

import sys



# Retrieve command-line arguments after '--'

argv = sys.argv

argv = argv[argv.index("--") + 1:]

if len(argv) < 2:

    print("Usage: blender --background --python fbx_to_obj.py -- <input_fbx> <output_obj>")

    sys.exit(1)



input_fbx = argv[0]

output_obj = argv[1]



# Remove all existing objects

bpy.ops.object.select_all(action='SELECT')

bpy.ops.object.delete()



# Import the FBX file

bpy.ops.import_scene.fbx(filepath=input_fbx)



# Optionally, apply transforms (if needed)

for obj in bpy.context.scene.objects:

    bpy.context.view_layer.objects.active = obj

    bpy.ops.object.transform_apply(location=True, rotation=True, scale=True)



# Export the scene as OBJ

bpy.ops.export_scene.obj(filepath=output_obj, use_selection=False)
